<a href="https://colab.research.google.com/github/kremerss/FOM-LaTeX-Template/blob/main/t5_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install pytorch_lightning
!pip install sentencepiece
!pip install datasets

In [ ]:
import pandas as pd
import datasets
from sklearn.datasets import fetch_20newsgroups

def build_20newsgroups_dataset(): #-> datasets.Dataset:
  newsgroups = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))

  df = pd.DataFrame(newsgroups.data, columns=['text'])
  df['label_id'] = pd.Series(data=newsgroups.target)

  df_target_names = pd.DataFrame(data=newsgroups.target_names, columns=['label'])
  df = pd.merge(df, df_target_names, left_on='label_id', right_index=True).sort_index()

  cat = {
    'religion': [0,15,19]
    ,'tech': list(range(1,6))
    ,'misc': [6]    
    ,'sports': list(range(7,11))
    ,'science': list(range(11,15))
    , 'politics': list(range(16,19))
  }

  for key, value in cat.items():
    df['label'].loc[df['label_id'].isin(value)] = key

  df.drop('label_id', axis=1, inplace=True)

  #return datasets.Dataset.from_pandas(df,preserve_index=False)
  return df

In [ ]:
  cat = {
    'religion': [0,15,19]
    ,'tech': list(range(1,6))
    ,'misc': [6]    
    ,'sports': list(range(7,11))
    ,'science': list(range(11,15))
    , 'politics': list(range(16,19))
  }

tokens = []
for key, value in cat.items():
    tokens.append(tokenizer(key))
max(len(token) for token in tokens)

2

In [ ]:
import sys
import argparse
import logging
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from datasets import Dataset as hugDataset
import pytorch_lightning as pl

from torch.optim import AdamW
from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split

In [ ]:
# Set up logging
logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.getLevelName("INFO"),
    handlers=[logging.StreamHandler(sys.stdout)],
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

In [ ]:
class customDataset(Dataset):
  def __init__(self, df, tokenizer, max_len=512):

    self.data = hugDataset.from_pandas(df, preserve_index=False)
    #print(self.data.head())
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.inputs = []
    self.targets = []

    self._build()

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, idx):
    source_ids = self.inputs[idx]['input_ids'].squeeze()
    target_ids = self.targets[idx]['input_ids'].squeeze()

    src_mask = self.inputs[idx]['attention_mask'].squeeze()
    trg_mask = self.targets[idx]['attention_mask'].squeeze()

    return {'source_ids': source_ids, 'target_ids': target_ids, 'source_mask': src_mask, 'target_mask': trg_mask}

  def _build(self):
    for idx in range(len(self.data)):
      input_, target = self.data['text'][idx], self.data['label'][idx]

      # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input_]
          , max_length=self.max_len
          , padding='max_length'
          , truncation=True
          , return_tensors="pt"
      )
      # tokenize targets
      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target]
          , max_length=2
          , padding='max_length'
          , truncation=True
          , return_tensors="pt"
      )
      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)

In [ ]:
class TestClass(pl.LightningModule):
  def __init__(self,df):
    logger.info('T5FineTuner init')
    super(TestClass, self).__init__()


    cuda = torch.device('cuda')
    self.df_train, self.df_test = train_test_split(df, test_size=.2)

a = TestClass(build_20newsgroups_dataset())

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
data = customDataset(a.df_train.head(),tokenizer=tokenizer)

data.__getitem__(0)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


{'source_ids': tensor([   27,   183,  1119,    12,  1431,    46,  1023,  1831,   478,    24,
          2284,     8,     3, 12604,  2471,  2594,  4924,     5,    37,  2471,
          2594,  5508,  2347, 19768,    11,  5223,    12,     8,   433,    28,
           150,   682,     5,   299,     8,   478, 22298,    44,     8,   166,
           580,    12,     3,     4, 10499,    51,   345,    76,    17, 29979,
             6,    28,     8,   826,  1569,    10,     3,     4,   848,    52,
           127,    13,  4567,  1690,    10,  3862, 10499,    51,   134,    15,
           122,    41,    77, 27769,  2471,  5508, 15577,    61,  9236,     3,
            32,   102,  4978,    13,  4567,  1690,    10,     3, 22974,    41,
         12604,    18,  9122,   329,    61, 20650,     3,    32,   102,  4978,
            13,  4567,  1690,    10,   220,    41,     4,   834, 10499,    51,
           345,    76,    17, 29979,    61, 15696,   297,     3,    23,    26,
            16,  4567,  1690,     3,  

In [ ]:
class T5FineTuner(pl.LightningModule):
  def __init__(self,df, hparams):
    logger.info('T5FineTuner init')
    super(T5FineTuner, self).__init__()


    cuda = torch.device('cuda')
    self.df_train, self.df_test = train_test_split(df, test_size=.2)
    self.hyparams = hparams
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name).to(device=cuda)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.model_name, model_max_length=hparams.max_len)#.to(device=cuda)

  def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
      # Function for performing the calculation of the module.
      logger.info('forward')
      return self.model(
          input_ids,
          attention_mask=attention_mask,
          decoder_input_ids=decoder_input_ids,
          decoder_attention_mask=decoder_attention_mask,
          labels=lm_labels,
      )

  def _step(self, batch):
      logger.info('_step')
      # Get target ids
      lm_labels = batch['target_ids']
      # Ignore padding token within the lost function  (-100 = ignore_index of the CrossEntropyLoss)
      lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

      # Generate inferences for the batch
      outputs = self(
          input_ids=batch['source_ids'],
          attention_mask=batch['source_mask'],
          lm_labels=lm_labels,
          decoder_attention_mask=batch['target_mask']
      )

      # Get batches loss
      loss = outputs[0]

      return loss

  def training_step(self,batch,batch_idx):
    logger.info('training_step')
    loss = self._step(batch)
    self.log('loss',loss)

    return({'loss':loss})

  def training_epochs_end(self,outputs):
    logger.info('training_epochs_end')
    avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
    self.log('avg_train_loss', avg_loss)

  def validation_step (self, batch, batch_idx):
    logger.info('validation_step')
    loss = self._step(batch)
    self.log('val_loss', loss)

  def configure_optimizers(self):
    logger.info('configure_optimizers')
    model = self.model
    no_decay = ['bias','LayerNorm.weight']

    optimizers_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)]
            , 'weight_decay': self.hyparams.weight_decay
        },
        {
            'params': [p for n,p in model.named_parameters() if any(nd in n for nd in no_decay)]
            , 'weight_decay': 0.0
        }
    ]

    optimizer = AdamW(optimizers_grouped_parameters
                      , lr=self.hyparams.learning_rate
                      , eps=self.hyparams.adam_epsilon)

    self.opt = optimizer

    return [optimizer]

  def optimizer_step(self, epoch=None, batch_idx=None, optimizer=None, optimizer_idx=None, optimizer_closure=None, on_tpu=None, using_native_amp=None, using_lbfgs=None):
    logger.info('optimizer_step')
    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
    
    lr = self.lr_scheduler.get_last_lr()[0]
    self.log('lr',lr)
    
  def _get_dataset(self, tokenizer, df, max_len):
    logger.info('_get_dataset')
    return customDataset(df=df, tokenizer=tokenizer, max_len=max_len)

  def train_dataloader(self):
    logger.info('train_dataloader')
    train_dataset = self._get_dataset(df=self.df_train, tokenizer=self.tokenizer, max_len=self.hyparams.max_len)
    dataloader = DataLoader(train_dataset, batch_size=self.hyparams.train_batch_size
                            , drop_last=True, shuffle=True, num_workers=4)

    dataset_len = len(dataloader.dataset)
    train_batch_mul_gpus = (
        self.hyparams.train_batch_size * max(1, self.hyparams.n_gpu))
    accumulate_grad_batches = self.trainer.accumulate_grad_batches
    train_epochs = float(self.hyparams.epochs)
    t_total = int((dataset_len // train_batch_mul_gpus) //
              accumulate_grad_batches * train_epochs)
    # Generate scheduler for linear
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hyparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler      
    return dataloader

  def val_dataloader(self):
    logger.info('val_dataloader')
    val_dataset = self._get_dataset(df=self.df_test, tokenizer=self.tokenizer, max_len=self.hyparams.max_len)
    return DataLoader(val_dataset, batch_size=self.hyparams.eval_batch_size
                      , num_workers=4)

In [ ]:
checkpoint = 't5-small'

hparams = dict(
    epochs=3
    , model_name=checkpoint
    , max_len=512
    , learning_rate=3e-4
    , weight_decay=0.0
    , adam_epsilon=1e-8
    , gradient_accumulation_steps=1
    , warmup_steps=0
    , train_batch_size=8
    , eval_batch_size=8
    , fp_16=False
    , opt_level='01' # for more information see https://nvidia.github.io/apex/amp.html#opt-levels
    , max_grad_norm=1.0
    , seed=42
    , n_gpu = 1
    )
model_args = argparse.Namespace(**hparams)
model = T5FineTuner(build_20newsgroups_dataset(),model_args)

In [ ]:
model.df_train.head()

,text,label
15505,What is hardware handshaking and when do I wan...,tech
13325,"\nAfter reading my local paper today, I found ...",sports
1070,Old Nick (the little devil!) wibbles:-\n%\n% T...,sports
16016,\n If the Anne Frank exhibit makes it to your...,religion
6596,\n\nIDE does not do DMA. This is because it's...,tech


In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-0ffaf8da-aba8-8627-aaa1-706856d9d796)


In [ ]:
args = argparse.Namespace(**hparams)

train_params = dict(
    #accumulate_grad_batches=args.gradient_accumulation_steps,
    accelerator='gpu',
    devices=1,
    max_epochs=args.epochs,
    #early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    amp_backend='apex',
    gradient_clip_val=args.max_grad_norm,
    #logger=tb_logger,
    #callbacks=lr_monitor,
    #enable_checkpointing=checkpoint_callback,
    #callbacks=[LoggingCallback()],
)

In [ ]:
trainer = pl.Trainer(**train_params)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
!mkdir /content/lightning_logs
trainer.fit(model)

NameError: ignored